In [1]:
from nltk import FreqDist
import numpy as np
import re

def buildDict(docs):
    doc_tokens = []     # python list
    for doc in docs:
        delim = re.compile(r'[\s,.]+')
        tokens = delim.split(doc.lower()) 
        if tokens[-1] == '' :   tokens = tokens[:-1] 
        doc_tokens.append(tokens)

    vocab = FreqDist(np.hstack(doc_tokens))
    vocab = vocab.most_common()
    word_to_id = {word[0] : id for id, word in enumerate(vocab)}
    id_to_word = {id : word[0] for id, word in enumerate(vocab)}
    return doc_tokens, vocab, word_to_id, id_to_word

In [2]:
docs = []
docs.append('To do is to be. To be is to do.')
docs.append('To be or not to be. I am what I am')
docs.append('I think therefore I am. Do be do be do.')
docs.append('Do do do da da da. Let it be let it be.')

doc_tokens, vocab, word_to_id, id_to_word = buildDict(docs)

In [3]:
from collections import Counter
import math

tf_vectors = []
for doc in doc_tokens:
    vec = [0.0 for _ in range((len(word_to_id)))] #사전 길이 리스트
    word_count = Counter(doc)     #단어별 문서 내 출현빈도
    for key, value in word_count.items():
        vec[word_to_id[key]] = 1+ math.log2(value) #tf계산
    tf_vectors.append(vec)

In [4]:
import pandas as pd

df = pd.DataFrame(tf_vectors, columns=id_to_word.values())
print(df)

         do   be   to    i   am        da   is  let   it   or  not  what  \
0  2.000000  2.0  3.0  0.0  0.0  0.000000  2.0  0.0  0.0  0.0  0.0   0.0   
1  0.000000  2.0  2.0  2.0  2.0  0.000000  0.0  0.0  0.0  1.0  1.0   1.0   
2  2.584963  2.0  0.0  2.0  1.0  0.000000  0.0  0.0  0.0  0.0  0.0   0.0   
3  2.584963  2.0  0.0  0.0  0.0  2.584963  0.0  2.0  2.0  0.0  0.0   0.0   

   think  therefore  
0    0.0        0.0  
1    0.0        0.0  
2    1.0        1.0  
3    0.0        0.0  


In [5]:
idf = {}
for id, _ in id_to_word.items():
    idf[id] = 0.0
    for doc in tf_vectors:
        if doc[id] > 0:
            idf[id] += 1

In [6]:
N = len(tf_vectors)
idf = {id : math.log2(N/val) for id, val in idf.items()}

In [7]:
df = pd.Series(idf.values(), index=idf.keys())
print(df)

0     0.415037
1     0.000000
2     1.000000
3     1.000000
4     1.000000
5     2.000000
6     2.000000
7     2.000000
8     2.000000
9     2.000000
10    2.000000
11    2.000000
12    2.000000
13    2.000000
dtype: float64


In [8]:
import numpy as np

idf_list = [val for _, val in idf.items()]
tfidf = np.array([np.multiply(tf, idf_list) for tf in tf_vectors])

In [9]:
df = pd.DataFrame(tfidf, columns=id_to_word.values())
df

,do,be,to,i,am,da,is,let,it,or,not,what,think,therefore
0,0.830075,0.0,3.0,0.0,0.0,0.000000,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,2.0,2.0,2.0,0.000000,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0
2,1.072856,0.0,0.0,2.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
3,1.072856,0.0,0.0,0.0,0.0,5.169925,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0


In [10]:
print(df.T)

                  0    1         2         3
do         0.830075  0.0  1.072856  1.072856
be         0.000000  0.0  0.000000  0.000000
to         3.000000  2.0  0.000000  0.000000
i          0.000000  2.0  2.000000  0.000000
am         0.000000  2.0  1.000000  0.000000
da         0.000000  0.0  0.000000  5.169925
is         4.000000  0.0  0.000000  0.000000
let        0.000000  0.0  0.000000  4.000000
it         0.000000  0.0  0.000000  4.000000
or         0.000000  2.0  0.000000  0.000000
not        0.000000  2.0  0.000000  0.000000
what       0.000000  2.0  0.000000  0.000000
think      0.000000  0.0  2.000000  0.000000
therefore  0.000000  0.0  2.000000  0.000000


In [11]:
from nltk import FreqDist
import numpy as np
import re

def buildDict(docs):
    doc_tokens = []     # python list
    for doc in docs:
        delim = re.compile(r'[\s,.]+')
        tokens = delim.split(doc.lower()) 
        if tokens[-1] == '' :   tokens = tokens[:-1] 
        doc_tokens.append(tokens)

    vocab = FreqDist(np.hstack(doc_tokens))
    vocab = vocab.most_common()
    word_to_id = {word[0] : id for id, word in enumerate(vocab)}
    id_to_word = {id : word[0] for id, word in enumerate(vocab)}
    return doc_tokens, vocab, word_to_id, id_to_word

In [12]:
from collections import Counter
import math
import numpy as np

def TFIDF(doc_tokens, id_to_word):
    tf_vectors = []
    idf = {}

    #TF 구하기
    for doc in doc_tokens:
        vec = [0.0 for _ in range((len(id_to_word)))]
        word_count = Counter(doc)
        for key, value in word_count.items():
            vec[word_to_id[key]] = value
            #vec[word_to_id[key]] = 1+ math.log2(value) #tf계산
        tf_vectors.append(vec)
    
    #IDF 구하기
    for id, _ in id_to_word.items():
        idf[id] = 0.0
        for doc in tf_vectors:
            if doc[id] > 0:
                idf[id] += 1
    N = len(tf_vectors)            
    idf = {id : np.log((N+1)/(val+1))+1 for id, val in idf.items()}

    #TF-IDF 구하기
    idf_list = [val for _, val in idf.items()]
    tfidf = np.array([np.multiply(tf, idf_list) for tf in tf_vectors])

    return tf_vectors, idf, tfidf

In [13]:
docs = []
docs.append('To do is to be. To be is to do.')
docs.append('To be or not to be. I am what I am')
docs.append('I think therefore I am. Do be do be do.')
docs.append('Do do do da da da. Let it be let it be.')

doc_tokens, vocab, word_to_id, id_to_word = buildDict(docs)
tf_vectors, idf, tfidf = TFIDF(doc_tokens, id_to_word)

In [14]:
import pandas as pd

pd.DataFrame(tfidf, columns=word_to_id.keys())

,do,be,to,i,am,da,is,let,it,or,not,what,think,therefore
0,2.446287,2.0,6.043302,0.000000,0.000000,0.000000,3.832581,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,2.0,3.021651,3.021651,3.021651,0.000000,0.000000,0.000000,0.000000,1.916291,1.916291,1.916291,0.000000,0.000000
2,3.669431,2.0,0.000000,3.021651,1.510826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.916291,1.916291
3,3.669431,2.0,0.000000,0.000000,0.000000,5.748872,0.000000,3.832581,3.832581,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
tfidf_l1 = np.array([vec / np.sum(vec) for vec in tfidf]) #l1 norm
pd.DataFrame(tfidf_l1, columns=word_to_id.keys())

,do,be,to,i,am,da,is,let,it,or,not,what,think,therefore
0,0.170804,0.139644,0.421954,0.000000,0.000000,0.000000,0.267598,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.118950,0.179712,0.179712,0.179712,0.000000,0.000000,0.000000,0.000000,0.113971,0.113971,0.113971,0.000000,0.000000
2,0.261458,0.142506,0.000000,0.215302,0.107651,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.136542,0.136542
3,0.192283,0.104803,0.000000,0.000000,0.000000,0.301249,0.000000,0.200833,0.200833,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
tfidf_l2 = np.array([vec / (np.sum(vec ** 2) ** 0.5) for vec in tfidf]) #l2 norm
pd.DataFrame(tfidf_l2, columns=word_to_id.keys())

,do,be,to,i,am,da,is,let,it,or,not,what,think,therefore
0,0.312717,0.255666,0.772535,0.000000,0.000000,0.000000,0.489931,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.307120,0.464005,0.464005,0.464005,0.000000,0.000000,0.000000,0.000000,0.294266,0.294266,0.294266,0.000000,0.000000
2,0.609695,0.332310,0.000000,0.502063,0.251031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.318401,0.318401
3,0.410533,0.223758,0.000000,0.000000,0.000000,0.643179,0.000000,0.428786,0.428786,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
from numpy.linalg import norm

tfidf_l2 = np.array([np.divide(vec , norm(vec)) for vec in tfidf]) #l2 norm
pd.DataFrame(tfidf_l2, columns=word_to_id.keys())

,do,be,to,i,am,da,is,let,it,or,not,what,think,therefore
0,0.312717,0.255666,0.772535,0.000000,0.000000,0.000000,0.489931,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.307120,0.464005,0.464005,0.464005,0.000000,0.000000,0.000000,0.000000,0.294266,0.294266,0.294266,0.000000,0.000000
2,0.609695,0.332310,0.000000,0.502063,0.251031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.318401,0.318401
3,0.410533,0.223758,0.000000,0.000000,0.000000,0.643179,0.000000,0.428786,0.428786,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:

user = input('질의어 입력')
delim = re.compile(r'[\s,.]+')
tokens = delim.split(user.lower()) 
if tokens[-1] == '' :   tokens = tokens[:-1] 
tokens

['to', 'do']

In [19]:
user_vector = np.array([0.0 for _ in vocab])
for token in tokens:
    if token in word_to_id:
        user_vector[word_to_id[token]] = idf[word_to_id[token]]
user_vector = np.array([np.divide(user_vector , norm(user_vector))]) #l2 norm
user_vector

array([[0.62922751, 0.        , 0.77722116, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [20]:
pd.DataFrame(tfidf_l2, columns=word_to_id.keys())

,do,be,to,i,am,da,is,let,it,or,not,what,think,therefore
0,0.312717,0.255666,0.772535,0.000000,0.000000,0.000000,0.489931,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.307120,0.464005,0.464005,0.464005,0.000000,0.000000,0.000000,0.000000,0.294266,0.294266,0.294266,0.000000,0.000000
2,0.609695,0.332310,0.000000,0.502063,0.251031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.318401,0.318401
3,0.410533,0.223758,0.000000,0.000000,0.000000,0.643179,0.000000,0.428786,0.428786,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
user_rank = [user_vector.dot(vec) for vec in tfidf_l2]
user_rank

[array([0.7972005]),
 array([0.3606342]),
 array([0.38363656]),
 array([0.25831867])]

In [22]:
cos_sim = np.array([np.dot(tfidf_l2, vector) for vector in tfidf_l2])
cos_sim

array([[1.        , 0.43698001, 0.27562227, 0.1855881 ],
       [0.43698001, 1.        , 0.45149814, 0.06872066],
       [0.27562227, 0.45149814, 1.        , 0.32465691],
       [0.1855881 , 0.06872066, 0.32465691, 1.        ]])

In [23]:
import re
import numpy as np
from nltk import FreqDist
from collections import Counter
import math

# 문장 리스트 정의
sentence = [
    'Python is a high-level, general-purpose programming language.',
    'Its design philosophy emphasizes code readability with the use of significant indentation.',
    'Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small- and large-scale projects.',
    'Python is dynamically-typed and garbage-collected.',
    'It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.',
    'It is often described as a "batteries included" language due to its comprehensive standard library.',
    'Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0.',
    'Python 2.0 was released in 2000 and introduced new features such as list comprehensions, cycle-detecting garbage collection, reference counting, and Unicode support.',
    'Python 3.0, released in 2008, was a major revision that is not completely backward-compatible with earlier versions.',
    'Python 2 was discontinued with version 2.7.18 in 2020.',
    'Python consistently ranks as one of the most popular programming languages',
]

def buildDict(docs):
    doc_tokens = []
    for doc in docs:
        delim = re.compile(r'[\s,.]+')
        tokens = delim.split(doc.lower())
        if tokens[-1] == '':
            tokens = tokens[:-1]
        doc_tokens.append(tokens)

    vocab = FreqDist(np.hstack(doc_tokens))
    vocab = vocab.most_common()
    word_to_id = {word[0]: id for id, word in enumerate(vocab)}
    id_to_word = {id: word[0] for id, word in enumerate(vocab)}
    return doc_tokens, vocab, word_to_id, id_to_word

def TFIDF(doc_tokens, id_to_word):
    tf_vectors = []
    idf = {}

    # TF 계산
    for doc in doc_tokens:
        vec = [0.0 for _ in range(len(id_to_word))]
        word_count = Counter(doc)
        for key, value in word_count.items():
            vec[word_to_id[key]] = value  # 단순 빈도 사용
        tf_vectors.append(vec)

    # IDF 계산
    for id, _ in id_to_word.items():
        idf[id] = 0.0
        for doc in tf_vectors:
            if doc[id] > 0:
                idf[id] += 1
    N = len(tf_vectors)
    idf = {id: np.log((N + 1) / (val + 1)) + 1 for id, val in idf.items()}

    # TF-IDF 계산
    idf_list = [val for _, val in idf.items()]
    tfidf = np.array([np.multiply(tf, idf_list) for tf in tf_vectors])

    return tf_vectors, idf, tfidf

def cosine_similarity(tfidf):
    # 코사인 유사도 계산
    cos_sim = np.zeros((len(tfidf), len(tfidf)))
    for i in range(len(tfidf)):
        for j in range(len(tfidf)):
            if i == j:
                cos_sim[i][j] = 1.0
            else:
                vec1 = tfidf[i]
                vec2 = tfidf[j]
                dot_product = np.dot(vec1, vec2)
                norm1 = np.linalg.norm(vec1)
                norm2 = np.linalg.norm(vec2)
                if norm1 == 0 or norm2 == 0:
                    cos_sim[i][j] = 0.0
                else:
                    cos_sim[i][j] = dot_product / (norm1 * norm2)
    return cos_sim

# 데이터 처리
doc_tokens, vocab, word_to_id, id_to_word = buildDict(sentence)
tf_vectors, idf, tfidf = TFIDF(doc_tokens, id_to_word)
cos_sim = cosine_similarity(tfidf)

# 결과 출력
np.set_printoptions(precision=4, suppress=True)
print("Cosine Similarity Matrix:")
print(cos_sim)

Cosine Similarity Matrix:
[[1.     0.     0.0551 0.1996 0.1323 0.1982 0.1827 0.0288 0.1541 0.0403
  0.1203]
 [0.     1.     0.1    0.     0.     0.0523 0.0701 0.     0.0476 0.0568
  0.1398]
 [0.0551 0.1    1.     0.1033 0.1065 0.1148 0.0853 0.0829 0.     0.
  0.    ]
 [0.1996 0.     0.1033 1.     0.062  0.0759 0.0987 0.1293 0.108  0.0463
  0.0497]
 [0.1323 0.     0.1065 0.062  1.     0.0492 0.1072 0.0498 0.     0.
  0.0917]
 [0.1982 0.0523 0.1148 0.0759 0.0492 1.     0.1712 0.0305 0.0715 0.
  0.0458]
 [0.1827 0.0701 0.0853 0.0987 0.1072 0.1712 1.     0.2149 0.1889 0.0892
  0.2035]
 [0.0288 0.     0.0829 0.1293 0.0498 0.0305 0.2149 1.     0.1476 0.2003
  0.0555]
 [0.1541 0.0476 0.     0.108  0.     0.0715 0.1889 0.1476 1.     0.1577
  0.0234]
 [0.0403 0.0568 0.     0.0463 0.     0.     0.0892 0.2003 0.1577 1.
  0.0279]
 [0.1203 0.1398 0.     0.0497 0.0917 0.0458 0.2035 0.0555 0.0234 0.0279
  1.    ]]
